In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Data Mensuelle

In [3]:
df_monthly = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df_monthly = df_monthly.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df_monthly.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df_monthly.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df_monthly.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap = market_cap.loc[df_48ind.index]


df_BtoM = df_monthly.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

# Data Journalière

In [6]:
df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)
df_daily = df_daily.rename(columns={'Unnamed: 0': 'Date'})

df_daily_ret = df_daily.iloc[:25670].copy()
df_daily_ret['Date'] = pd.to_datetime(df_daily_ret['Date'], format='%Y%m%d')
df_daily_ret.set_index('Date', inplace=True)
df_daily_ret = df_daily_ret.apply(pd.to_numeric, errors='coerce')
df_daily_ret.replace(-99.99, np.nan, inplace = True)
df_daily_ret.replace(-999, np.nan, inplace = True)
df_daily_ret.dropna(inplace = True)


df_FF = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/F-F_Research_Data_Factors_daily.CSV', header=3)
df_FF = df_FF.rename(columns={'Unnamed: 0': 'Date'})
df_FF_daily = df_FF.iloc[:25670].copy()
df_FF_daily['Date'] = pd.to_datetime(df_FF_daily['Date'], format='%Y%m%d')
df_FF_daily.set_index('Date', inplace=True)
df_FF_daily = df_FF_daily.apply(pd.to_numeric, errors='coerce')
df_FF_daily.replace(-99.99, np.nan, inplace = True)
df_FF_daily.replace(-999, np.nan, inplace = True)
df_FF_daily.dropna(inplace = True)
df_FF_daily = df_FF_daily.loc[df_daily_ret.index]

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1260/2405719038.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)


In [7]:
df_daily_ret

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,0.54,0.55,-0.19,0.43,0.84,0.90,1.15,-0.04,0.69,0.38,...,0.29,0.81,0.39,0.44,1.69,0.22,0.30,1.43,0.03,3.69
1969-07-02,0.89,1.27,0.51,1.02,0.95,2.47,0.35,0.28,0.83,0.48,...,1.71,0.76,1.27,0.08,1.31,1.53,0.72,1.71,0.12,-0.04
1969-07-03,-0.17,0.69,0.57,1.49,-0.02,1.49,0.51,0.61,0.75,0.65,...,1.01,0.68,0.96,0.13,1.22,2.39,-0.21,1.36,1.27,2.58
1969-07-07,1.27,-0.65,-0.61,0.42,-0.26,-1.09,-1.84,0.30,-0.52,-0.44,...,-0.47,-1.21,-0.42,-0.52,-0.57,-0.75,-1.52,-2.29,-0.85,-2.26
1969-07-08,-0.71,-0.56,-0.94,-0.21,-1.36,-2.02,-2.36,-0.66,-1.38,-1.77,...,-1.39,-1.35,-1.36,-1.28,-2.66,-0.92,-1.39,-1.87,-1.44,-1.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-25,-0.08,0.87,0.35,0.86,0.48,1.33,2.14,0.82,1.90,0.43,...,2.30,1.67,0.67,0.74,0.08,0.52,-1.63,1.80,0.78,0.91
2024-01-26,1.13,0.55,0.29,0.67,-0.28,1.11,0.76,0.11,0.59,1.59,...,0.23,-0.46,-0.31,0.76,-0.08,0.42,0.74,0.24,0.02,0.84
2024-01-29,0.04,0.52,0.54,0.22,0.90,0.80,0.81,1.08,0.56,1.13,...,-0.65,0.76,0.70,0.81,1.11,0.87,-0.02,1.42,0.46,-0.29


1950-2023, 1990-2023.... permet de comparer dans le temps

In [ ]:
Ri_t = df_daily_ret['Agric']
Rft = df_FF_daily['RF']
Rm_t = df_FF_daily['Mkt-RF']

Y = Ri_t - Rft
X = Rm_t

X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()

print(results.summary())

In [16]:
Rft = df_FF_daily['RF']
Rm_t = df_FF_daily['Mkt-RF']

betas = []  # Create an empty list to store the betas

for column in df_daily_ret.columns:
    Ri_t = df_daily_ret[column]

    Y = Ri_t - Rft
    X = Rm_t

    X = sm.add_constant(X)

    model = sm.OLS(Y, X)
    results = model.fit()

    betas.append(results.params[1])  # Add the betas to the list

    #print(f"Results for column {column}:")
    #print(results.summary())
    #print("\n---\n")

# Convert the list of betas to a numpy array
betas = np.array(betas)

In [17]:
betas

array([0.76049147, 0.6393648 , 0.75541845, 0.67901135, 0.68172561,
       1.05355646, 1.19578685, 0.93095544, 0.78784488, 0.98827778,
       0.93246416, 0.89300155, 0.8480455 , 1.01690703, 0.8571561 ,
       0.9676043 , 1.02997989, 1.19403685, 1.25401259, 1.01815488,
       1.13479216, 1.1543125 , 1.17432549, 1.03883184, 0.88860026,
       0.71312572, 0.37313947, 1.08769315, 1.21856869, 0.95604899,
       0.60350605, 0.86499256, 0.93649497, 1.12478546, 1.22420941,
       1.28295055, 1.1118323 , 0.87343067, 0.92430643, 1.00871137,
       0.88952992, 0.94924313, 0.92565506, 1.14232427, 0.94384186,
       1.00438329, 1.21445785, 0.95600125])

In [24]:
# Définir la date de début et de fin
start_date = '1969-07-01'  # Remplacez par la date de début souhaitée
end_date = '1970-07-01'  # Remplacez par la date de fin souhaitée

# Sélectionner les données pour la plage de dates spécifiée
df_daily_ret_selected = df_daily_ret.loc[start_date:end_date]
df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

# Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
Rft = df_FF_daily_selected['RF']
Rm_t = df_FF_daily_selected['Mkt-RF']

betas = []  # Create an empty list to store the betas

for column in df_daily_ret_selected.columns:
    Ri_t = df_daily_ret_selected[column]

    Y = Ri_t - Rft
    X = Rm_t

    X = sm.add_constant(X)

    model = sm.OLS(Y, X)
    results = model.fit()

    betas.append(results.params[1])  # Add the betas to the list

    #print(f"Results for column {column}:")
    #print(results.summary())
    #print("\n---\n")

# Convert the list of betas to a numpy array
betas = np.array(betas)

In [25]:
betas

array([0.82921422, 0.68240304, 0.86016423, 0.75758158, 0.83108174,
       1.44655584, 1.81838221, 0.84245923, 1.0761161 , 1.03253307,
       2.35402225, 1.34107248, 0.97141749, 0.90491647, 1.10651056,
       0.87356122, 1.13295647, 1.29759374, 1.02031329, 1.02039316,
       0.98071617, 1.0650064 , 1.00116715, 1.39303999, 0.95999824,
       1.13240883, 1.05736242, 1.12659462, 1.26227225, 0.99547757,
       0.54552442, 0.61527212, 1.76691909, 1.34854126, 1.31422145,
       1.41354227, 1.27187625, 1.01541069, 0.87560574, 1.21212954,
       1.5333847 , 0.86745787, 1.58361965, 0.93976056, 1.24645115,
       1.9405922 , 1.22401651, 1.5132537 ])